# WosClient

In [1]:
import time
import pandas as pd

In [2]:
# !pip install suds-jurko

from suds import client
from base64 import b64encode as _b64encode
from collections import OrderedDict as _OrderedDict

class WosClient():
    """Query the Web of Science.
       You must provide user and password only to user premium WWS service.
       with WosClient() as wos:
           results = wos.search(...)"""

    base_url = 'http://search.webofknowledge.com'
    auth_url = base_url + '/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
    search_url = base_url + '/esti/wokmws/ws/WokSearch?wsdl'
    searchlite_url = base_url + '/esti/wokmws/ws/WokSearchLite?wsdl'

    def __init__(self, user=None, password=None, SID=None, close_on_exit=True,
                 lite=False):
        """Create the SOAP clients. user and password for premium access."""

        self._SID = SID
        self._close_on_exit = close_on_exit
        search_wsdl = self.searchlite_url if lite else self.search_url
        self._auth = client.Client(self.auth_url)
        self._search = client.Client(search_wsdl)

        if user and password:
            auth = '%s:%s' % (user, password)
            auth = _b64encode(auth.encode('utf-8')).decode('utf-8')
            headers = {'Authorization': ('Basic %s' % auth).strip()}
            self._auth.set_options(headers=headers)

    def __enter__(self):
        """Automatically connect when used with 'with' statements."""
        self.connect()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Close connection after closing the 'with' statement."""
        if self._close_on_exit:
            self.close()

    def __del__(self):
        """Close connection when deleting the object."""
        if self._close_on_exit:
            self.close()

    def connect(self):
        """Authenticate to WOS and set the SID cookie."""
        if not self._SID:
            self._SID = self._auth.service.authenticate()
            print('Authenticated (SID: %s)' % self._SID)

        self._search.set_options(headers={'Cookie': 'SID="%s"' % self._SID})
        self._auth.options.headers.update({'Cookie': 'SID="%s"' % self._SID})
        return self._SID

    def close(self):
        """Close the session."""
        if self._SID:
            self._auth.service.closeSession()
            self._SID = None

    def search(self, query, count=5, offset=1):
        """Perform a query. Check the WOS documentation for v3 syntax."""
        if not self._SID:
            raise RuntimeError('Session not open. Invoke .connect() before.')

        qparams = _OrderedDict([('databaseId', 'WOS'),
                                ('userQuery', query),
                                ('queryLanguage', 'en')])

        rparams = _OrderedDict([('firstRecord', offset),
                                ('count', count),
                                ('sortField', _OrderedDict([('name', 'RS'),
                                                            ('sort', 'D')]))])

        return self._search.service.search(qparams, rparams)

In [3]:

from xml.etree import ElementTree as _ET
from xml.dom import minidom as _minidom
import re as _re

def single(wosclient, wos_query, xml_query=None, count=10, offset=1):
    """Perform a single Web of Science query and then XML query the results."""
    result = wosclient.search(wos_query, count, offset)
    print (result)
    xml = _re.sub(' xmlns="[^"]+"', '', result.records, count=1).encode('utf-8')
    if xml_query:
        xml = _ET.fromstring(xml)
        return [el.text for el in xml.findall(xml_query)]
    else:
        return _minidom.parseString(xml).toprettyxml()

def query(wosclient, wos_query, xml_query=None, count=10, offset=1, limit=100):
    """Query Web of Science and XML query results with multiple requests."""
    results = [single(wosclient, wos_query, xml_query, min(limit, count-x+1), x) for x in range(offset, count+1, limit)]
    if xml_query:
        return [el for res in results for el in res]
    else:
        pattern = _re.compile(r'.*?<records>|</records>.*', _re.DOTALL)
        return ('<?xml version="1.0" ?>\n<records>' +
                '\n'.join(pattern.sub('', res) for res in results) +
                '</records>')

def doi_to_wos(wosclient, doi):
    """Convert DOI to WOS identifier."""
    results = query(wosclient, 'DO=%s' % doi, './REC/UID', count=1)
    return results[0].lstrip('WOS:') if results else None

In [4]:
soap = WosClient(lite=True)
soap.connect()

Authenticated (SID: D3BtfMB8s7cppbOs7cg)


D3BtfMB8s7cppbOs7cg

# Exchange the Given and Family Name

In [5]:
searchnames = []
i=0
for line in open('Dismis_Acad_List_CLEAN.txt','r'):
    QueryString = line.strip()
    QueryString = "AU="+line.split(',')[0].split()[-1]+' '+line.split(',')[0].split()[0]
    print('QueryString:', QueryString)
    searchnames.append(QueryString)


QueryString: AU=AYDIN AYDIN
QueryString: AU=GİLGİL ERDAL
QueryString: AU=HALAÇ METİN
QueryString: AU=YÜCE ABDULHAKİM
QueryString: AU=KIRIŞ ABDULKADİR
QueryString: AU=KÜÇÜKBAYRAK ABDULKADİR
QueryString: AU=YILDIRIM ABDULKADİR
QueryString: AU=ÇEKİN ABDULKADİR
QueryString: AU=ŞENGÜN ABDULKADİR
QueryString: AU=BEDİR ABDULKERİM
QueryString: AU=AKPINAR ABDULLAH
QueryString: AU=BAL ABDULLAH
QueryString: AU=ŞENGÜL ABDULLAH
QueryString: AU=DEMİR ABDULLAH
QueryString: AU=DOĞAN ABDULLAH
QueryString: AU=EKİZER ABDULLAH
QueryString: AU=ERDİL ABDULLAH
QueryString: AU=ERGİN ABDULLAH
QueryString: AU=SEÇKİN ABDULLAH
QueryString: AU=KIRAN ABDULLAH
QueryString: AU=ÜSTÜN ABDULLAH
QueryString: AU=CİNGÜ ABDULLAH
QueryString: AU=AKSOY ABDULLAH
QueryString: AU=TUNCER ABDULLAH
QueryString: AU=MUTLU ABDULLAH
QueryString: AU=KILDAN ABDULLAH
QueryString: AU=POŞ ABDULLAH
QueryString: AU=SÖNMEZ ABDULLAH
QueryString: AU=URAL ABDULLAH
QueryString: AU=YÖRDEM ABDULLAH
QueryString: AU=TARIKULU ABDULLAH
QueryString: AU=Ç

QueryString: AU=ÇATLIOĞLU HAKAN
QueryString: AU=ÇETİN HAKAN
QueryString: AU=ÇETİNEL HAKAN
QueryString: AU=ÖZHAN HAKAN
QueryString: AU=ÖZKÖKELİ HAKAN
QueryString: AU=SARI HAKAN
QueryString: AU=ŞAHİN HAKAN
QueryString: AU=ŞANDIR HAKAN
QueryString: AU=FINDIK HAKKI
QueryString: AU=GÖKBEL HAKKI
QueryString: AU=SARNEL HALDUN
QueryString: AU=BORAZAN HALE
QueryString: AU=SAVAŞ HALUK
QueryString: AU=CEYLAN HALUK
QueryString: AU=SÖYLEMEZ HALÜK
QueryString: AU=PEKTEZEL HALİD
QueryString: AU=AKMAN HALİL
QueryString: AU=AYKUL HALİL
QueryString: AU=BEYDİLLİ HALİL
QueryString: AU=BİLECEN HALİL
QueryString: AU=KUNT HALİL
QueryString: AU=ÖZLER HALİL
QueryString: AU=ŞEN HALİL
QueryString: AU=TOSUN HALİL
QueryString: AU=UÇAL HALİL
QueryString: AU=YILDIRIM HALİL
QueryString: AU=ÇALIŞKAN HALİL
QueryString: AU=ÇETİN HALİL
QueryString: AU=ÖZGÜNER HALİL
QueryString: AU=BAHAR HALİL
QueryString: AU=DEMİRCİ HALİL
QueryString: AU=ESKİKURT HALİL
QueryString: AU=HALİLOĞLU HALİL
QueryString: AU=KILINÇ HALİL
QueryStr

QueryString: AU=BUCAKTEPE PAKİZE
QueryString: AU=BAYRAM PARVANA
QueryString: AU=YALÇINOĞLU PELİN
QueryString: AU=ULUS PERVİN
QueryString: AU=SEKMEN PERİHAN
QueryString: AU=ECEVİTOĞLU PINAR
QueryString: AU=ÖZUĞUZ PINAR
QueryString: AU=ARSLAN RAMAZAN
QueryString: AU=BALCI RAMAZAN
QueryString: AU=BAŞTÜRK RAMAZAN
QueryString: AU=BOZKURT RAMAZAN
QueryString: AU=BÜYÜKKAYA RAMAZAN
QueryString: AU=DERDİMAN RAMAZAN
QueryString: AU=DAVRAN RAMAZAN
QueryString: AU=DURAN RAMAZAN
QueryString: AU=GÜLENDAM RAMAZAN
QueryString: AU=GÜN RAMAZAN
QueryString: AU=GÜNAY RAMAZAN
QueryString: AU=KARANFİL RAMAZAN
QueryString: AU=KAYIKCI RAMAZAN
QueryString: AU=KAYNAK RAMAZAN
QueryString: AU=KILIÇ RAMAZAN
QueryString: AU=KILIÇARSLAN RAMAZAN
QueryString: AU=KOCABAŞ RAMAZAN
QueryString: AU=KOZAN RAMAZAN
QueryString: AU=KURT RAMAZAN
QueryString: AU=MEMİŞOĞULLARI RAMAZAN
QueryString: AU=MERT RAMAZAN
QueryString: AU=TÜRKMEN RAMAZAN
QueryString: AU=USLU RAMAZAN
QueryString: AU=UYGUR RAMAZAN
QueryString: AU=YAĞCI RAMAZ

In [6]:
len(searchnames)

3575

In [7]:
pd.DataFrame(searchnames,columns=['SearchNames']).to_csv('SearchNames.csv')

In [8]:
SearchNames = pd.read_csv('SearchNames.csv')

In [9]:
SearchNames.SearchNames[0].strip().split('=')[1]

'AYDIN AYDIN'

In [47]:
names_tr = []
lens = []
co_aut = []
i=0
for i in range(len(SearchNames)):
#     if i > 10: break
    print('Will approximately take ' + str(len(SearchNames) / 2) + ' seconds..')
    line = SearchNames.SearchNames[i]
    QueryString = line
    print(QueryString)
    if i == 2400: # WOS constraints that each ID can only search for 2500 times. Just set smaller counts to renew the ID.
        soap = WosClient(lite=True)
        soap.connect()
    results = soap.search(QueryString)
    #getting number of name of co-authors
    if (results.recordsFound>0):
        authors = []
        for j in range (len(results.records)):
            for k in range (len(results.records[j].authors)):
                for l in range (len(results.records[j].authors[k].value)):
                    co_authors=results.records[j].authors[k].value[l]
                    authors.append(co_authors)
                    print('co_author',co_authors)
                #co_aut.append(authors)
            #co.append(co_aut)
    print(results.recordsFound)
    names_tr.append(QueryString.split('=')[1])
    lens.append(results.recordsFound)
    co_aut.append(authors)
    time.sleep(0.5) # WOS constraints that you can only search twice per second
    i=i+1

Will approximately take 1787.5 seconds..
AU=AYDIN AYDIN
co_author Kul, Sibel
co_author Aydin, Aydin
co_author Dinc, Hasan
co_author Erduran, Erol
co_author Aydin, Leyla
co_author Aydin, Aydin
co_author Bulbul, Baris
co_author Ozkalkan, Beliz
co_author Ozaydin, Cansu
co_author Inaltekin, Ezgi
co_author Aydin, Aydin
co_author Aksoy, Hamit Zafer
co_author Reis, Abdulkadir
co_author Ozdemir, Feyyaz
co_author Zengin, Ahmet Y.
co_author Topkan, Erkan
co_author Aydin, Aydin
co_author Reis, A. Kadir
4
Will approximately take 1787.5 seconds..
AU=GİLGİL ERDAL
co_author Melikoglu, Meltem Alkan
co_author Kocabas, Hilal
co_author Sezer, Ilhan
co_author Akdag, Ali
co_author Gilgil, Erdal
co_author Butun, Bulent
co_author Dundar, Umit
co_author Kupesiz, Alphan
co_author Ozdem, Sehahat
co_author Gilgil, Erdal
co_author Tuncer, Tiraje
co_author Yesilipek, Akif
co_author Gultekin, Meral
co_author Gilgil, Erdal
co_author Kacar, Cahit
co_author Tuncer, Tiraje
co_author Tuncer, Tiraje
co_author Gilgil, Erd

Will approximately take 1787.5 seconds..
AU=BAL ABDULLAH
co_author Alam, Mohammad S.
co_author Islam, Mohammed Nazrul
co_author Bal, Abdullah
co_author Karim, Mohammad A.
co_author Alam, Mohammad S.
co_author Bal, Abdullah
co_author Albora, A. Muhittin
co_author Bal, Abdullah
co_author Ucan, Osman N.
co_author Bal, Abdullah
co_author Uslu, Faruk Sukru
co_author Binol, Hamidullah
co_author Ilarslan, Mustafa
co_author Bal, Abdullah
26
Will approximately take 1787.5 seconds..
AU=ŞENGÜL ABDULLAH
co_author Emsley, Robin
co_author Hargarter, Ludger
co_author Bergmans, Paul
co_author Uglesic, Boran
co_author Sengul, Abdullah Cem
co_author Petralia, Antonino
co_author Khannanova, Angelina
co_author Cherubin, Pierre
co_author Schreiner, Andreas
co_author Karaoglan, Mustafa
co_author Tutar, Suleyman
co_author Sengul, Abdullah Murat
co_author Tekeli, Hakan
co_author Togrol, Rifat Erdem
co_author Sengul, Abdullah
3
Will approximately take 1787.5 seconds..
AU=DEMİR ABDULLAH
co_author Ulucanlar, Hal

Will approximately take 1787.5 seconds..
AU=GÖK ABDULVAHAP
co_author Erkutlu, Ibrahim
co_author Buyukhatipoglu, Hakan
co_author Alptekin, Mehmet
co_author Gok, Abdulvahap
co_author Erkutlu, Ibrahim
co_author Buyukhatipoglu, Hakan
co_author Alptekin, Mehmet
co_author Ozsarac, Coskun
co_author Buyukbese, Ilhan
co_author Gok, Abdulvahap
co_author Gezgin, Inan
co_author Erkutlu, Ibrahim
co_author Alptekin, Mehmet
co_author Geyik, Murat
co_author Gok, Abdulvahap
co_author Geyik, Murat
co_author Erkutlu, Ibrahim
co_author Alptekin, Mehmet
co_author Gezgin, Man
co_author Mizrak, Ayse
co_author Pusat, Serhat
co_author Gok, Abdulvahap
co_author Geyik, Murat
co_author Erkutlu, Ibrahim
co_author Alptekin, Mehmet
co_author Gezgin, Inan
co_author Mizrak, Ayse
co_author Dokur, Mehmet
co_author Gok, Abdulvahap
10
Will approximately take 1787.5 seconds..
AU=ÖZTÜRK ABDURAHMAN
0
Will approximately take 1787.5 seconds..
AU=ABAKAY ABDURRAHMAN
co_author Tanrikulu, A. Cetin
co_author Hosoglu, Salih
co_autho

Will approximately take 1787.5 seconds..
AU=BÖLÜKBAŞI ABDURREHİM
co_author Ciloglu, Dogan
co_author Bolukbasi, Abdurrehim
co_author Cifci, Harun
1
Will approximately take 1787.5 seconds..
AU=BÖREKCİ ABDURREZZAK
co_author Borekci, Abdurrezzak
co_author Gur, Mustafa
co_author Ucar, Hakan
co_author Seker, Taner
co_author Cayli, Murat
co_author Baykan, Ahmet Oytun
co_author Gur, Mustafa
co_author Kalkan, Gulhan Yuksel
co_author Acele, Armagan
co_author Seker, Taner
co_author Sen, Omer
co_author Borekci, Abdurrezzak
co_author Koc, Mevlut
co_author Cayli, Murat
co_author Akyol, Selahattin
co_author Cortuk, Mustafa
co_author Baykan, Ahmet Oytun
co_author Kiraz, Kemal
co_author Borekci, Abdurrezzak
co_author Seker, Taner
co_author Gur, Mustafa
co_author Cayli, Murat
co_author Borekci, Abdurrezzak
co_author Gur, Mustafa
co_author Turkoglu, Caner
co_author Selek, Sahbettin
co_author Baykan, Ahmet Oytun
co_author Seker, Taner
co_author Harbalioglu, Hazar
co_author Ozaltun, Betuel
co_author Makca,

Will approximately take 1787.5 seconds..
AU=KARACA ADEM
co_author Karkucak, Murat
co_author Cilesizoglu, Nurce
co_author Capkin, Erhan
co_author Can, Ipek
co_author Batmaz, Ibrahim
co_author Kerimoglu, Servet
co_author Onder, Mustafa Avni
co_author Karaca, Adem
co_author Ayar, Ahmet
co_author Kocak, Mustafa
co_author Civan, Nadim
co_author Karkucak, Murat
co_author Capkin, Erhan
co_author Garipoglu, Melek
co_author Can, Ipek
co_author Karaca, Adem
co_author Tasdelen, Mustafa
co_author Ayar, Ahmet
co_author Capkin, Erhan
co_author Karkucak, Murat
co_author Cosar, Arif M.
co_author Ak, Emel
co_author Karaca, Adem
co_author Gokmen, Ferhat
co_author Budak, Bayram S.
co_author Tosun, Mehmet
co_author Gokmen, Ferhat
co_author Akbal, Ayla
co_author Savas, Yilmaz
co_author Zateri, Coskun
co_author Karaca, Adem
co_author Tan, Yusuf Ziya
co_author Capkin, Erhan
co_author Karkucak, Murat
co_author Kola, Mehmet
co_author Karaca, Adem
co_author Capkin, Arzu Aydin
co_author Karahan, Suleyman Cancer


...

Will approximately take 1787.5 seconds..
AU=ERDURMUŞ MESUT
co_author Erdurmus, Mesut
co_author Selcoki, Yusuf
co_author Yagci, Ramazan
co_author Hepsen, Ibrahim F.
co_author Aydin, Bahri
co_author Cagil, Nurullah
co_author Erdogan, Selcuk
co_author Erdurmus, Mesut
co_author Hasiripi, Hikmet
co_author Erdurmus, Mesut
co_author Yagci, Ramazan
co_author Karadag, Remzi
co_author Durmus, Mustafa
co_author Erdurmus, Mesut
co_author Yagci, Ramazan
co_author Karadag, Remzi
co_author Durmus, Mustafa
co_author Totan, Yueksel
co_author Yagci, Ramazan
co_author Erdurmus, Mesut
co_author Bayrak, Reyhan
co_author Hepsen, Ibrahim Feyzi
40
Will approximately take 1787.5 seconds..
AU=GARİPARDIÇ MESUT
co_author Toktamis, Aydin
co_author Demirel, Yeltekin
co_author Ozkan, Keramettin Ugur
co_author Garipardic, Mesut
co_author Gozukucuk, Ali
co_author Nur, Naim
co_author Anak, Sema S.
co_author Atay, Didem
co_author Garipardic, Mesut
co_author Ozalp, Berit
co_author Yani, Diana
co_author Agaaoglu, Leyla
co

Will approximately take 1787.5 seconds..
AU=DOĞAN METİN
co_author Dogan, Metin
co_author Feyzioglu, Bahadir
co_author Oezdemir, Mehmet
co_author Baysal, Buelent
co_author Bozkurt, Murat
co_author Ugurlu, Mahmut
co_author Dogan, Metin
co_author Tosun, Nihat
co_author Bozkurt, Murat
co_author Isik, Cetin
co_author Gursoy, Safa
co_author Akkaya, Mustafa
co_author Algin, Oktay
co_author Dogan, Metin
co_author Ozer, Turkan Toka
co_author Yula, Erkan
co_author Dogan, Metin
co_author Baskin, Huseyin
co_author Cetinkol, Yeliz
co_author Aladal, Murat
co_author Unal, Nevzat
co_author Bayramoglu, Gulcin
co_author Ahmetbayik, Seyit
co_author Dogan, Metin
co_author Durmaz, Suleyman
co_author Gunes, Hayati
co_author Ozlemkalayci, Hacer
co_author Cetinonarer, Pelin
co_author Mengeloglu, Zafer
co_author Milletli, Fikriye
co_author Ozdemir, Metin
co_author Yula, Erkan
co_author Keremcalgin, M.
co_author Yanik, Keramettin
60
Will approximately take 1787.5 seconds..
AU=DURU METİN
co_author Duru, Metin
co

Will approximately take 1787.5 seconds..
AU=KURT MEVLÜT
co_author Kurt, Mevlut
co_author Disibeyaz, Selcuk
co_author Akdogan, Meral
co_author Sasmaz, Nurgul
co_author Aksu, Salih
co_author Haznedaroglu, Ibrahim Celalettin
co_author Kurt, Mevlut
co_author Harputluoglu, Hakan
co_author Dede, Didem S.
co_author Gullu, Ibrahim H.
co_author Altundag, Kadri
co_author Kurt, Mevlut
co_author Shorbagi, Ali
co_author Aksu, Salih
co_author Haznedaroglu, Ibrahim
co_author Altundag, Kadri
co_author Erkin, Gul
co_author Kurt, Mevlut
co_author Altundag, Kadri
co_author Kurt, Mevlut
co_author Aksoy, Sercan
co_author Cizginer, Sevdenur
co_author Harmanci, Ozgur
co_author Altundag, Kadri
co_author Kakaman, Nazmiye
106
Will approximately take 1787.5 seconds..
AU=YAPRAK MEVLÜT
co_author Altun, Gulay Durmus
co_author Vardar, Selma Arzu
co_author Turan, Nesrin
co_author Yaprak, Mevlut
1
Will approximately take 1787.5 seconds..
AU=ÇELİKOĞLU MEVLÜT
co_author Celikoglu, Mevlut
co_author Buyuk, Suleyman Kutalmi

Will approximately take 1787.5 seconds..
AU=GÜL MUAMMER


WebFault: Server raised fault: '(ISE0002) Exception occurred processing request. Reason: Error received from a server supporting the Web service. Cause: Supporting server error message = [ID limit 2500 exceeded]. Supporting server error code = [Server.IDLimit]. Remedy: If this is a problem with your request, correct your request and submit it again. Otherwise, call customer support. '

In [53]:
#a python module to make unicode strings work as expected for turkish chars. solves the turkish "İ" problem

# have to do pip install unicode_tr
names_en = []
from unicode_tr.extras import slugify
for i in range(len(names_tr)):
    name_en = slugify(names_tr[i])
    name_en = name_en.replace('-',' ').title()
    names_en.append(name_en)

In [58]:
len(names_tr)

1926

In [59]:
cnt=0
for l in co_aut:
    cnt += len(l)
print('Totally ' + str(cnt) + ' guys were downloaded as coauthors')

Totally 44265 guys were downloaded as coauthors


In [69]:
coauthors = set()
for l in co_aut:
    for author in l:
        author = author.replace(',', '')
        coauthors.add(author)

In [73]:
# let's now create the file with not-dismissed professors:
dismissed = set(names_en)

print('Totally we have ' + str(len(dismissed)) + ' dismissed and ' + str(len(coauthors)) + ' unique coauthors')

Totally we have 1892 dismissed and 17338 unique coauthors


In [75]:
undismissed = coauthors - dismissed
print('So, we have ' + str(len(undismissed)) + ' undismissed guys')

So, we have 16059 undismissed guys


In [76]:
pd.Dataframe(list(dismissed)).to_csv('dismissed.csv')
pd.Dataframe(list(undismissed)).to_csv('undismissed.csv')

AttributeError: module 'pandas' has no attribute 'to_csv'